### Week 4 - Homework

In [18]:
import pandas as pd 
import numpy as np 
import sys, os

from sklearn.metrics import roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

Data Preperation

In [3]:
cols = ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style',
       'highway MPG', 'city mpg', 'MSRP']

data = pd.read_csv(r'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv')[cols]

In [4]:
data.columns = data.columns.str.lower().str.replace(' ', '_')
data.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'msrp'],
      dtype='object')

In [5]:
data.isnull().any()

make                 False
model                False
year                 False
engine_hp             True
engine_cylinders      True
transmission_type    False
vehicle_style        False
highway_mpg          False
city_mpg             False
msrp                 False
dtype: bool

In [6]:
data = data.fillna(0)
data.isnull().any()

make                 False
model                False
year                 False
engine_hp            False
engine_cylinders     False
transmission_type    False
vehicle_style        False
highway_mpg          False
city_mpg             False
msrp                 False
dtype: bool

In [7]:
msrp_mean = data['msrp'].mean()
data['above_average'] = data['msrp'].apply(lambda x: 1 if x > msrp_mean else 0)

Split Dataset

In [41]:
y = data.above_average.values 
X = data.drop(['msrp', 'above_average'], axis=1)

X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=1)

#### Question 1: ROC AUC feature importance

ROC AUC could also be used to evaluate feature importance of numerical variables.

For each numerical variable, use it as score and compute AUC with the above_average variable. Use the training dataset for this task.

If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. -df_train['engine_hp'])

AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

In [42]:
numerical_cols = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

auc_dict = {}

for col in numerical_cols:
    auc_score = roc_auc_score(y_train, X_train[col].values)
    
    if auc_score < 0.5:
        auc_score = roc_auc_score(y_train, -X_train[col].values)

    auc_dict[col] = auc_score 



In [43]:
print(f"The numerical variable that has the highest AUC: {max(auc_dict, key= lambda x: auc_dict[x])}")

The numerical variable that has the highest AUC: engine_hp


#### Question 2: Training the model

Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

In [44]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1_000)
dv = DictVectorizer(sparse=False)

In [48]:
X_train_enc = dv.fit_transform(X_train.to_dict(orient='records'))

model.fit(X_train_enc, y_train)

X_val_enc = dv.transform(X_val.to_dict(orient='records'))

y_pred_val = model.predict_proba(X_val_enc)[:, 1]

auc_score = round(roc_auc_score(y_val, y_pred_val), 3)

In [49]:
print(f"The AUC score is: {auc_score}")

The AUC score is: 0.979


#### Question 3: Precision and Recall

Compute precision and recall for the model.

Evaluate the model on all thresholds from 0.0 to 1.0 with step 0.01
For each threshold, compute precision and recall

In [54]:
from sklearn.metrics import recall_score, precision_score

thresholds = np.linspace(0, 1, 101)
precision_scores = []
recall_scores = []

for t in thresholds:
    actual_positive = (y_val == 1)
    actual_negative = (y_val == 0)

    predict_positive = (y_pred_val >= t)
    predict_negative = (y_pred_val < t)

    
    

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36
0.37
0.38
0.39
0.4
0.41000000000000003
0.42
0.43
0.44
0.45
0.46
0.47000000000000003
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.96
0.97
0.98
0.99
1.0
